In vanilla RNN, we compute only hidden state. But in LSTM along with hidden state we also comopute another state which we call cell state $c_t$. Intuitively it can be thought as $c_t$ stores long short term information. LSTM can read, write, erase infromation to and from this $c_t$ cell.

![alt text](images/lstm0.PNG)

So we can say, in hidden layer of LSTM, we have now input vector $x_t$, previous hidden state $h_{t-1}$, and previous cell state $c_{t-1}$. From these three vectors we will now compute, current hidden state $h_t$ and current cell state $c_t$ at a particular time step t as follows.

$\begin{aligned}
f_t &= \sigma(W_{hf}h_{t_1} + W_{xf}x_t) \\
i_t &= \sigma(W_{hi}h_{t_1} + W_{xi}x_t) \\
o_t &= \sigma(W_{ho}h_{t_1} + W_{xo}x_t) \\
g_t &= \text{tanh}(W_{hg}h_{t_1} + W_{xg}x_t) \\
\end{aligned}$


![alt text](images/lstm1.PNG)![alt text](images/lstm2.PNG) 


![alt text](images/lstm4.PNG)



Form the above images and illustration we can describe LSTM and its working procedures as follows:

After stacking the input and previous hidden state size becomes 2h. Since we will calculate 4 gates, each gate will need a chunk of weights to multiply with input vector.Therefore the size of the weight is "4hX2h". After calculation i,f,o will pass through sigmoid gate and g gate will go through tnah. That means i,f,o will be a vectors containing values between 0 and 1. On the other hand the g gate will contain values between -1 and 1. Before going further let's talk about the gates.

- **f-forget gate**: controls how much information need to be removed from the previous steps which is why LSTM has two state $c_t$ and $h_t$. This $c_t$ will propagate over time and learn whether to forget the previous cell state or not.

- **i-input gate**: controls how much information needs to be "added" tho the next cell state from previous hidden state $h_{t-1}$ and $x_t$. Since i has values between 0 and 1, this serves as switch. This gate decides whether to take the RNN output that is produced by gate g and multiply the output with input gate i.

- **o-output gate**: controls how much information need to be shown as output in the current hidden state $h_t$.

All these above illustrations can be shown in a single picture easily.




![alt text](images/lstm5.PNG)

**Summary**: The three softmax gates i,f,o controls calculations by their almost 0/1 properties. 0 means, value at that position will not be considered and vice versa.

Cell state have these different independent scalar values which are being incremented or decremented by 1. That means inside the cell state we can either remember or forget our previous state, and then we can either increment or decrement each element of that cell state. Therefore we can kind of say these elements as integer counters.

### Do LSTM solve the vanishing gradient problem? 
The LSTM architecture makes it easier for the RNN to preserve information over many timesteps.
- e.g. if the f=1, i=0, then information of that cell state is preserved.
- By contrast, it's hard for vanilla RNN to learn a recurrent weight matrix Wh that preserves info in hidden state.

Well, LSTM doesn't gurantee that there is no vanishing gradient or gradient exploding. But it does provide the model an easier way for the model to learn long-distance dependencies which we will be visualised in the gradient flow of LSTM.

#### Graident flow of LSTM
Now when we backpropagate through the addition operation the upstream gradient is copied into two branches. Then this gradient is passed through element wise multiplication with the forget gate. That means the only thing that happens to our upstream gradient is that it ends up getting multiplied elemenetwise by the forget gate. This is better than vanilla RNN in several ways-

- This forget gate is now elementwise multiplication rather than matrix multiplication
- This elementwise multiplication is multiplying by different forget get at each time step. In vanilla RNN we continually multiply by the same weight matrix which leads to gradient exploding/ vanishing gradient.
- The forget gate comes out from sigmoid gate. Therefore this elementwise multiplication guranteed to be between zero and one which is good thing in terms of multiplying over and over again.
- Last but not the least, In vanilla RNN, gradient pass through a tanh every time. But in LSTM, hidden state is used to compute those outputs $y_t$. So now if we imagine backprpagating from the final hidden state back to the firt cell state, then through that backward pass we only backpropagate through only a single "tanh" non linearity rather than through a separate tanh at every time step.


**Note**: If the forget gates are always less than one, then vanishing gradient can happen in LSTM likewise vanilla RNN. To avoid this situation one trick is to initializes the biases of the forget gate to be somewhat positive. So at the begining of training those forget gates are always very close to one.


![alt text](images/lstmgf.PNG)

#### Gradient of loss w.r.t. W:
In vanilla RNN, we end up adding those first time step w gradients to compute our final gradient on w. In LSTM, at every time step we will take our current hidden state as well as current cell state which will give us our local gradient on w for that time step. So, when we have long sequence we are getting gradients to the very end of the sequence. As we backpropagate through, we  will get local gradients on w for each time step and that local gradient on w will be coming through these gradients on "v" and "h". Because we are monitoring gradients on c much more nicely in LSTM, those local gradients on w at each time step  will also be carried forward and backward through time.